Project 2: Scraping apartment sales prices 

Name: Veera Tegelberg
Email: veera.tegelberg@gmail.com
GitHub: TegelVee

This project is part of Columbia University's Lede Program 2024. In the project I will be scraping housing prices in Helsinki metropolitan area from Asuntojen hintatiedot webpage: https://asuntojen.hintatiedot.fi/haku/. The page gathers sales prices from real estate brokers and is maintained by a public office.

Data limitations
- the data does not encompass all houses that have been sold
- sales date is not reported, so shortterm comparison is not possible
- information stays on the page for 12 months, so neather longterm comparison is possible
- data does not include postalcodes, only name of the are, which is as an information less trustworthy than postalcode

Due to these limitations, data must be seen as directional snapshot of local housing market

PART 1: SCRAPING THE DATA

In [49]:
#Bring libraries that I need.
import requests
import pandas as pd
from bs4 import BeautifulSoup
from random import randint 
from time import sleep


In [50]:
#scrape the first webpage and turn it into a bs4 object

url = "https://asuntojen.hintatiedot.fi/haku/?c=Helsinki&cr=1&t=3&l=2&z=1&search=1&sf=0&so=a&renderType=renderTypeTable&print=0&submit=Next+page"
html = requests.get(url).text
soup = BeautifulSoup(html)
#soup

In [51]:
# Base URL with a placeholder for the 'z' parameter
base_url = "https://asuntojen.hintatiedot.fi/haku/?c=Helsinki&cr=1&t=3&l=2&z={}&search=1&sf=0&so=a&renderType=renderTypeTable&print=0&submit=Next+page"

collected_data = []

# Set range from 1 to 48 to go through all tabs that hold information about house prices.
for z in range(1, 48):
    # Format the URL with the current value of 'z'
    url = base_url.format(z)
    
    # Send an HTTP request to the URL
    # The URL is formated with base_url and by changing the z with different number while for looping
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract the data from table rows (tr)
        # This does not collect the headerline
        table_rows = soup.find_all('tr')
        
        # Here information of each row is put into a list with for loop
        for row in table_rows:
            # td referring to each table cell
            columns = row.find_all('td')
            # Taking all whitespaces away with strip
            data = [column.get_text(strip=True) for column in columns]
            # append needed to get all rows into a one list
            collected_data.append(data)

    # Precaution just in case the data is not found        
    else:
        print(f"Failed to retrieve page {z}")

    # A delay, so the server will not be overloaded
    # It takes around 2 minutes to collect the data
    import time
    time.sleep(2)

print("Collected data:")
for data in collected_data:
    print(data)

Collected data:
[]
['1', '', '1', '']
['', '1', '']
['Studio apartment']
['Itä-pasila', '1h,kt,alkovi', 'kt', '34,00', '283000', '8324', '2020', '3/6', 'yes', 'good', 'own', 'C2013']
['Kallio', '1h+k+alkovi+kph', 'kt', '37,50', '276000', '7360', '1926', '4/7', 'yes', 'good', 'own', 'F2013']
['Tapaninvainio', '1h, kk, kh, ter...', 'kt', '24,00', '64000', '2667', '1964', '1/2', 'no', 'average', 'own', 'G2013']
['Kallio', '1h, kk, kph, vh', 'kt', '28,00', '170000', '6071', '1962', '4/7', 'yes', 'good', 'own', 'D2018']
['Hakaniemi', '1h+k', 'kt', '36,00', '190000', '5278', '1989', '2/8', 'yes', '', 'own', 'F2018']
['Taka-töölö', '1h, avok, kph', 'kt', '20,50', '156000', '7610', '1939', '1/6', 'yes', 'good', 'own', 'D2018']
['Lauttasaari', '1h+kk+vh+kph+la...', 'kt', '25,50', '215000', '8431', '1961', '2/5', 'yes', 'good', 'own', 'C2018']
['Kallio', '1h, avok, kph, wc', 'kt', '26,60', '160000', '6015', '1929', '1/6', 'yes', 'good', 'own', 'D2018']
['Kallio', '1h+kk', 'kt', '29,00', '158500'

In [52]:
# Now, I want the headers too.
# I need them only from the first page.

header = soup.select("thead th")
# header

# I call the headers 'column_names' 
# Finding them with list comprehension
column_names = [ cell.text.strip() for cell in header ]
#print it out
column_names

# I want also substitude " ◄" with nothing "", so I need to do some cleaning.
# I make a new variable and use lambda to replace ' ◄'.
# 'map' means that lambda function is used in every list member.
# 'list' returns result into a list again.
# .strip cleans extra whitespaces away.
cleaned_headers = list(map(lambda x: x.replace(' ◄', '').strip(), column_names))

cleaned_headers


['District',
 'Apartment',
 'Type of building',
 'm2',
 'Debt-free price',
 '€/m2',
 'Construction year',
 'Floor',
 'Elevator',
 'Condition',
 'Plot',
 'Energy class']

In [53]:
# Let's turn 'column_names' and 'collected_data' into a pandas dataframe

df = pd.DataFrame(collected_data, columns = cleaned_headers)
df

,District,Apartment,Type of building,m2,Debt-free price,€/m2,Construction year,Floor,Elevator,Condition,Plot,Energy class
0,None,None,None,None,None,None,None,None,None,None,None,None
1,1,,1,,None,None,None,None,None,None,None,None
2,,1,,None,None,None,None,None,None,None,None,None
3,Studio apartment,None,None,None,None,None,None,None,None,None,None,None
4,Itä-pasila,"1h,kt,alkovi",kt,"34,00",283000,8324,2020,3/6,yes,good,own,C2013
...,...,...,...,...,...,...,...,...,...,...,...,...
2426,,5 h+k+p+s+at+ku...,ok,"120,00",352000,2933,1956,,no,poor,own,
2427,,"5h, k, rt, 2 kp...",ok,"154,00",485000,3149,1950,,no,average,rent,D2018
2428,,4h+k+2kph+khh+s...,ok,"128,00",455000,3555,2005,2/2,no,good,own,B2018
2429,47,,47,,None,None,None,None,None,None,None,None


In [54]:
# Let's do couple routine checkings.

# Check the data types
df.dtypes

# All data types are still objects. Need to change them later.

District             object
Apartment            object
Type of building     object
m2                   object
Debt-free price      object
€/m2                 object
Construction year    object
Floor                object
Elevator             object
Condition            object
Plot                 object
Energy class         object
dtype: object

In [55]:
len(df)
# My data has 2431 rows in the beginng. 
# This matches with number seen in down left corner of 'df' (see above). 

2431

PART 2: CLEANING THE DATA

Now, I have the data of apartments that have been registered into the service during last 12 months.
My data was gathered on July 20, 2024.

To be able to use the data, I need to clean it first. Following steps need to be done:
- Change 'Debt free price' into integer.
- Remove rows on which column 'Debt free price' is 'None'. This eliminates rows that do not include interesting information.
- If 'Apartment' starts with '1', make a new column 'Apartment_type' and write down there 'studio'. 
- Do the same thing with 'Apartment' with 2 (two-room), 3 (three-room), 4 (four-room), >=5 (five-room or bigger)
- 

Save all changes into 'df_cleaned'. 
Check number of row, len(df_cleaned).


In [56]:
# I want to check from how many rows 'Debt-free price' is missing, meaning that it equals 'None'.

missing_price = df[df['Debt-free price'].isna()]
# missing_price.head()

print(f'Debt-free price is missing from {len(missing_price)} rows.')

Debt-free price is missing from 275 rows.


In [57]:
df1 = df.copy()

In [58]:
# I remove all rows with 'None' in 'Debt-free price' and save new df in 'df_cleaned'.

df1 = df.dropna(subset=['Debt-free price'])
df1

,District,Apartment,Type of building,m2,Debt-free price,€/m2,Construction year,Floor,Elevator,Condition,Plot,Energy class
4,Itä-pasila,"1h,kt,alkovi",kt,"34,00",283000,8324,2020,3/6,yes,good,own,C2013
5,Kallio,1h+k+alkovi+kph,kt,"37,50",276000,7360,1926,4/7,yes,good,own,F2013
6,Tapaninvainio,"1h, kk, kh, ter...",kt,"24,00",64000,2667,1964,1/2,no,average,own,G2013
7,Kallio,"1h, kk, kph, vh",kt,"28,00",170000,6071,1962,4/7,yes,good,own,D2018
8,Hakaniemi,1h+k,kt,"36,00",190000,5278,1989,2/8,yes,,own,F2018
...,...,...,...,...,...,...,...,...,...,...,...,...
2418,,5h+k+saunaosast...,ok,"130,00",390000,3000,1997,-1/2,no,good,own,
2425,,"5h,k,pesuh,s,kh...",ok,"108,00",320000,2963,1990,-1/2,no,average,rent,E2018
2426,,5 h+k+p+s+at+ku...,ok,"120,00",352000,2933,1956,,no,poor,own,
2427,,"5h, k, rt, 2 kp...",ok,"154,00",485000,3149,1950,,no,average,rent,D2018


In [59]:
# I check, if 'dropna' deleted right amount of rows.
# Result should be 2156 = 2431 (rows in df) - 275 (rows with 'None').

print(f' There are {len(df)-len(missing_price)} rows in the new data frame "df1".')

 There are 2156 rows in the new data frame "df1".


In [60]:
# On the webpage squaremeters are displayed with the Finnish way (32,00). Comma needs to be changed into dot (32.00).

df1.loc[:, 'm2'] = df1['m2'].str.replace(",", ".")
df1.head()


,District,Apartment,Type of building,m2,Debt-free price,€/m2,Construction year,Floor,Elevator,Condition,Plot,Energy class
4,Itä-pasila,"1h,kt,alkovi",kt,34.00,283000,8324,2020,3/6,yes,good,own,C2013
5,Kallio,1h+k+alkovi+kph,kt,37.50,276000,7360,1926,4/7,yes,good,own,F2013
6,Tapaninvainio,"1h, kk, kh, ter...",kt,24.00,64000,2667,1964,1/2,no,average,own,G2013
7,Kallio,"1h, kk, kph, vh",kt,28.00,170000,6071,1962,4/7,yes,good,own,D2018
8,Hakaniemi,1h+k,kt,36.00,190000,5278,1989,2/8,yes,,own,F2018


In [61]:
# I name a new variable that contains changes done above.
df2 = df1.copy()

In [62]:
# I change four columns containing numbers into int or float, depending how the numbers are formed in df above.
df2["Debt-free price"] = df2["Debt-free price"].astype(int)
df2["m2"] = df2["m2"].astype(float)
df2["Construction year"] = df2["Construction year"].astype(int)
df2["€/m2"] = df2["€/m2"].astype(int)


# Checking, if changes went through.
print(df2.dtypes)

District              object
Apartment             object
Type of building      object
m2                   float64
Debt-free price        int32
€/m2                   int32
Construction year      int32
Floor                 object
Elevator              object
Condition             object
Plot                  object
Energy class          object
dtype: object


In [63]:
# Checking, how does the df now look like.
df2.tail()

,District,Apartment,Type of building,m2,Debt-free price,€/m2,Construction year,Floor,Elevator,Condition,Plot,Energy class
2418,,5h+k+saunaosast...,ok,130.0,390000,3000,1997,-1/2,no,good,own,
2425,,"5h,k,pesuh,s,kh...",ok,108.0,320000,2963,1990,-1/2,no,average,rent,E2018
2426,,5 h+k+p+s+at+ku...,ok,120.0,352000,2933,1956,,no,poor,own,
2427,,"5h, k, rt, 2 kp...",ok,154.0,485000,3149,1950,,no,average,rent,D2018
2428,,4h+k+2kph+khh+s...,ok,128.0,455000,3555,2005,2/2,no,good,own,B2018


In [64]:
# I want to divide 'Apartment' and form a new column called 'Apartment_type'.
# Rule for the new column: 
# If 'Apartment' starts with '1', write to the new column 'studio'.
# If it starts with '2' --> 'two-room', '3' --> 'three-room', '4' --> 'four-room', '>=5' --> 'five-rooms or bigger'

# To create the new column with conditions based on existing column, I import numpy library.
import numpy as np

# I create a list of our conditions based on which number the 'Apartment' starts.
conditions = [
    df2['Apartment'].str.startswith('1'),
    df2['Apartment'].str.startswith('2'),
    df2['Apartment'].str.startswith('3'),
    df2['Apartment'].str.startswith('4'),
]

# Here are new values for each conditions. These will be set into 'Apartment_type'.
choices = ['studio', 'two-rooms', 'three-rooms', 'four-rooms']

# I make classification 'five-rooms or bigger'.
# It will be given in 'Apartment_type' for any row not matching 'conditions'.
df2['Apartment_type'] = np.select(conditions, choices, default='five-rooms or bigger')

df2

,District,Apartment,Type of building,m2,Debt-free price,€/m2,Construction year,Floor,Elevator,Condition,Plot,Energy class,Apartment_type
4,Itä-pasila,"1h,kt,alkovi",kt,34.0,283000,8324,2020,3/6,yes,good,own,C2013,studio
5,Kallio,1h+k+alkovi+kph,kt,37.5,276000,7360,1926,4/7,yes,good,own,F2013,studio
6,Tapaninvainio,"1h, kk, kh, ter...",kt,24.0,64000,2667,1964,1/2,no,average,own,G2013,studio
7,Kallio,"1h, kk, kph, vh",kt,28.0,170000,6071,1962,4/7,yes,good,own,D2018,studio
8,Hakaniemi,1h+k,kt,36.0,190000,5278,1989,2/8,yes,,own,F2018,studio
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2418,,5h+k+saunaosast...,ok,130.0,390000,3000,1997,-1/2,no,good,own,,five-rooms or bigger
2425,,"5h,k,pesuh,s,kh...",ok,108.0,320000,2963,1990,-1/2,no,average,rent,E2018,five-rooms or bigger
2426,,5 h+k+p+s+at+ku...,ok,120.0,352000,2933,1956,,no,poor,own,,five-rooms or bigger
2427,,"5h, k, rt, 2 kp...",ok,154.0,485000,3149,1950,,no,average,rent,D2018,five-rooms or bigger


In [65]:
# Checking how many different 'Apartment_type' there are. 
# Most common is two-room, most rare five-rooms or bigger. This sounds plausible, so I continue.
df2.Apartment_type.value_counts()

Apartment_type
two-rooms               755
three-rooms             583
studio                  374
four-rooms              305
five-rooms or bigger    139
Name: count, dtype: int64

In [67]:
# I want to check are there in my 'five-rooms or bigger' classification apartments from wrong categories.

df2['Apartment'] = df2['Apartment'].astype(str)

# I don't want on my checklist rows starting with numbers, only the wrong ones that start with something else.
# I create variable that I can use to exclude all right answers.
# I just guess, there are no apartments bigger than 12-rooms, so I make variable including numbers from 5 to 12.
prefixes = ('5', '6', '7', '8', '9', '10', '11', '12')

checklist = df2.loc[
    (df2['Apartment_type'] == 'five-rooms or bigger') & 
    # '~' turns my command meaning opposite than I am writing
    # This is why code below returns rows, that DO NOT start with any 'prefixes' given above.
    (~df2['Apartment'].apply(lambda x: any(str(x).startswith(prefix) for prefix in prefixes))),
    'Apartment'
]

print(f'There are {len(checklist)} apartments that do not fulfill definition of 5-room or bigger apartments.')
print("-----------")
print(f'Here are the cases: {checklist}')
print("-----------")


There are 17 apartments that do not fulfill definition of 5-room or bigger apartments.
-----------
Here are the cases: 132      Studio, avok, kph
142     huone, avokeitt...
179     i h, k, kh (Yj ...
239         huone, kk, kph
1177    Oh, avok, mh, k...
1494      Oh+avokeitti...
1891         Yläkerta: ...
1903      AK: k/oh, ty...
1907    ak: oh, avokeit...
1921    oh, k, 5 x mh, ...
2092    Oh, kt, 3mh, vh...
2203    Ak: 2h, k, ruok...
2230    Olohuone, keitt...
2314    Lapsi- ja perhe...
2321    Yk: 2mh, kph, h...
2400    Oh, avok, 4 mh,...
2412    Kerrostalo, 20 ...
Name: Apartment, dtype: object
-----------


In [68]:
# I make a new dataframe 'df3' that do not contain 'checklist' rows.
df3 = df2[~df2['Apartment'].isin(checklist)]
df3.head()

,District,Apartment,Type of building,m2,Debt-free price,€/m2,Construction year,Floor,Elevator,Condition,Plot,Energy class,Apartment_type
4,Itä-pasila,"1h,kt,alkovi",kt,34.0,283000,8324,2020,3/6,yes,good,own,C2013,studio
5,Kallio,1h+k+alkovi+kph,kt,37.5,276000,7360,1926,4/7,yes,good,own,F2013,studio
6,Tapaninvainio,"1h, kk, kh, ter...",kt,24.0,64000,2667,1964,1/2,no,average,own,G2013,studio
7,Kallio,"1h, kk, kph, vh",kt,28.0,170000,6071,1962,4/7,yes,good,own,D2018,studio
8,Hakaniemi,1h+k,kt,36.0,190000,5278,1989,2/8,yes,,own,F2018,studio


In [69]:
# Checking, if the code really dropped 17 rows off my df.
len(df3) - len(df2)

-17

In [72]:
# Are there rows that are totally equal? 
# This is important to know, so that I can be sure, there are no duplicates. 
# It would be weird to have exatly same kind of apartments in the df, because there are so many parameters in the data.
# If there are duplicates, leave only one row.

# Are there duplicates? True = yes.
duplicates = df3.duplicated()
print(duplicates.value_counts())

print('-----------')

# There is one duplicate. How does it look like?
duplicate_row = df3[df3.duplicated()]
print('This is how the duplicate row looks like:')
print(duplicate_row)
print(f'There are {len(df3)} rows in df3.')

print('-----------')

# I delete it and make a new dataframe 'df_cleaned'.
df_cleaned = df3.drop_duplicates()

# Checking that there is now one row less.
print(f'There are {len(df_cleaned)} rows in the df_cleaned.')



False    2138
True        1
Name: count, dtype: int64
-----------
This is how the duplicate row looks like:
         District           Apartment Type of building    m2  Debt-free price  \
2376  Lauttasaari  4-5h, k, kph, w...               kt  96.0           655000   

      €/m2  Construction year Floor Elevator Condition Plot Energy class  \
2376  6823               2017   2/7      yes      good  own        C2013   

     Apartment_type  
2376     four-rooms  
There are 2139 rows in df3.
-----------
There are 2138 rows in the df_cleaned.


Now the data is enough clean for analyzing.
One cleaning thing could be still to drop out rows missing 'District'. For now, I want to keep them, because I want to ask questions about whole Helsinki, so the district does not matter yet.

PART 3: INTERVIEWING THE DATA

Interesting questions to ask:
1) Whole city
- Most expensive and cheapest apartments (Debt-free price)?
- Average square meter price (€/m2) in Helsinki?
- Smallest and biggest apartment?
- How many apartments have been sold? 
- What are the Type of buildings? How many 

2) District
- How many districts there are?
- How many apartments have been sold from each district?
- Which districts (top 10) are most represented in the data? = From which areas apartments have been sold most, according to this data?
- New dataframe: 'Apartment_type' per distict? --> I want to compare where there have been studios vs. big apartments sold?

3) Notice:
It would be nice to make a map out of this, but it will be difficult, because Datawrapper's choropleth map holds different district division (boroughs vs. here mishmash created by real estate brokers). --> If I clean data well, I can try.
Map
- €/m2 by district --> Comparing neighboring boroughs
- Flats sold by area --> tooltip about studio, two-rooms, three-rooms etc.

In [74]:
# Most expensive and cheapest apartments (Debt-free price)?

most_exp_aprt = df_cleaned["Debt-free price"].max()
print(f'Most expensive apartment has costed {most_exp_aprt} euros.')

print('- - - - - - - - - -')

cheapest_apart = df_cleaned["Debt-free price"].min()
print(f'Most cheapest apartment has costed {cheapest_apart} euros.')

print('- - - - - - - - - -')

# How many times more the most expesive apartment has costed vs. the cheapest?
print(f'The most expensive apartments sold in Helsinki costed {round(most_exp_aprt/cheapest_apart)} times as much as the most cheapest apartment sold.')

print('- - - - - - - - - -')

# Where were these apartments located?

max_price_index = df_cleaned['Debt-free price'].idxmax()
max_price_district = df_cleaned.loc[max_price_index, 'District']
print(f'The most expensive apartment was sold in {max_price_district}.')

print('- - - - - - - - - -')

min_price_index = df_cleaned['Debt-free price'].idxmin()
min_price_district = df_cleaned.loc[min_price_index, 'District']
print(f'The cheapest apartment was located in {min_price_district}.')

print('- - - - - - - - - -')

# How big were they?

max_price_size = df_cleaned.loc[max_price_index, 'm2']
print(f'The size of the most expensive apartment was {max_price_size} m2.')

print('- - - - - - - - - -')

min_price_size = df_cleaned.loc[min_price_index, 'm2']
print(f'The size of the cheapest apartment was {min_price_size} m2.')

# Print the whole data row.
min_price_row = df_cleaned.loc[min_price_index]
max_price_row = df_cleaned.loc[max_price_index]

print('- - - - - - - - - -')

print('HERE IS ALL INFORMATION ABOUT THE MOST EXPENSIVE:')
print(max_price_row)

print('- - - - - - - - - -')

print('HERE IS ALL INFORMATION ABOUT THE CHEAPEST:')
print(min_price_row)

Most expensive apartment has costed 2100000 euros.
- - - - - - - - - -
Most cheapest apartment has costed 43000 euros.
- - - - - - - - - -
The most expensive apartments sold in Helsinki costed 49 times as much as the most cheapest apartment sold.
- - - - - - - - - -
The most expensive apartment was sold in Ullanlinna.
- - - - - - - - - -
The cheapest apartment was located in Koskela.
- - - - - - - - - -
The size of the most expensive apartment was 201.5 m2.
- - - - - - - - - -
The size of the cheapest apartment was 42.5 m2.
- - - - - - - - - -
HERE IS ALL INFORMATION ABOUT THE MOST EXPENSIVE:
District                       Ullanlinna
Apartment              6h, k, kph, spa...
Type of building                       kt
m2                                  201.5
Debt-free price                   2100000
€/m2                                10422
Construction year                    1915
Floor                                -1/5
Elevator                               no
Condition             

It would be nice to write a story about the cheapest and the most expensive apartments, but I do not have time for mapping. 
I decide instead to check out, have the apartments become smaller during last decades. In Finland there has been heavy discussion that construction companies are all the time building smaller and smaller apartments. For example, there can be new three-room apartments with less than 50 m2. These are criticized for being unpractical to furniture and to live in.
Let's see, what is the average size of a studio, a two-room apartment and a three-room apartment built on different decades.
I do some sorting, because I do not want to check every decade as a separate section. By grouping the 'Construction year' I get more results per category.

SORTING:
-1929,
1930-1949,
1950-1969,
1970-1989,
1990-2009,
2010-

In [83]:
# I create a variable 'relevant_apartments' and take in only studios, two-rooms and three rooms.
relevant_apartments = df_cleaned[df_cleaned['Apartment_type'].isin(['studio', 'two-rooms', 'three-rooms'])].copy()

# Defining categories by 'Construction year'.
def categorize_year(year):
    if year <= 1929:
        return '1929 or earlier'
    elif 1930 <= year <= 1949:
        return '1930-1949'
    elif 1950 <= year <= 1969:
        return '1950-1969'
    elif 1970 <= year <= 1989:
        return '1970-1989'
    elif 1990 <= year <= 2009:
        return '1990-2009'
    else:
        return '2010 or later'

# Making a new column 'Construction decade'.
relevant_apartments.loc[:, 'Construction decade'] = relevant_apartments['Construction year'].apply(categorize_year)

# With .agg() function (=aggregation) it is possibel to apply multiple functions to the grouped data.
# I use it, because I want to see the median size (m2) and number of data points by category.
grouped = round(relevant_apartments.groupby(['Construction decade', 'Apartment_type']).agg(
    median_m2=('m2', 'median'),
    count=('m2', 'size')
).reset_index())

print(grouped)

   Construction decade Apartment_type  median_m2  count
0      1929 or earlier         studio       32.0    107
1      1929 or earlier    three-rooms       80.0     44
2      1929 or earlier      two-rooms       53.0     86
3            1930-1949         studio       26.0     56
4            1930-1949    three-rooms       64.0     31
5            1930-1949      two-rooms       44.0    112
6            1950-1969         studio       28.0    130
7            1950-1969    three-rooms       67.0    186
8            1950-1969      two-rooms       50.0    253
9            1970-1989         studio       34.0     55
10           1970-1989    three-rooms       75.0    163
11           1970-1989      two-rooms       55.0    158
12           1990-2009         studio       38.0      8
13           1990-2009    three-rooms       77.0     74
14           1990-2009      two-rooms       52.0     68
15       2010 or later         studio       31.0     18
16       2010 or later    three-rooms       69.0

In [84]:
# Save results into excel file.
output_file = 'apartment_analysis.xlsx'
grouped.to_excel(output_file, index=False)